# TP Observations Nocturnes - LU3PY232
**ARGUELLO Camilo**

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.visualization import astropy_mpl_style
plt.style.use(astropy_mpl_style)
from utils import pc2m, m2pc, dms2deg, deg2dms, flux2mag, mag2flux, distance_modulus, apparent_to_absolute_flux
from constants import ConstantesAstro
from matplotlib.colors import LogNorm
from astropy.stats import sigma_clip
import os 
import glob
#import astroalign as aa
from photutils.aperture import aperture_photometry, CircularAperture
from photutils.centroids import centroid_quadratic
import processing as  pr
from IPython.display import display, Math

## 1. Introduction et Contexte

### Objectifs du TP

Ce TP vise à étudier la relation d’échelle entre la taille et la luminosité des galaxies de l’univers proche. 
Nous allons utiliser des images obtenues à l'aide de télescopes à distance pour :

- Calibrer des images photométriques (offset, flat field),
- Extraire des informations scientifiques (taille, luminosité),
- Comparer les résultats avec les théories d’assemblage galactique.


### Diversité morphologique des galaxies

// TODO

### Loi d'échelle taille-luminosité

// TODO

//  explication des relations taille-masse et leur interprétation physique


### Messier 83
// TODO

## 2. Prise en main et visualisation des images de M83

> Quel est l’ordre de grandeur de la brillance de surface des etoiles ?  la brillance de surface du fond de ciel ? La brillance de surface de la galaxie ? En quelle unit´e est le flux sur ces images non-calibr´ees ?

In [ ]:
# Données fournies
FOCALLEN = 2912            # distance focale en mm
XPIXSZ = 9e-3              # taille du pixel en mm (9 microns)
distance_Mpc = 4.66        # distance depuis la Terre en Mpc
angular_size_arcsec = 600  # taille angulaire mesurée en arcsec

# Calcul du plate scale théorique (en arcsec par pixel)
plate_scale = 206265 * (XPIXSZ / FOCALLEN)
print(f"Plate scale théorique : {plate_scale:.3f} arcsec/pixel")

# Conversion de la taille angulaire en taille physique
# Conversion d'arcsec à radian : 1 rad = 206265 arcsec
angular_size_rad = angular_size_arcsec / 206265
distance_pc = distance_Mpc * 1e6  # conversion de Mpc à pc
size_pc = angular_size_rad * distance_pc
size_kpc = size_pc / 1000

print(f"Taille physique de la galaxie : {size_pc:.2f} pc ({size_kpc:.2f} kpc)")

# Estimation approximative de la masse de la galaxie
# Pour une galaxie spirale typique, une taille de ~13-15 kpc correspond souvent à une masse de l'ordre de 10^11 Msun.
# Cette estimation grossière repose sur des relations empiriques et reste à préciser avec d'autres mesures.
estimated_mass = 1e11  # masse en masse solaire (Msun)
print(f"Masse estimée de la galaxie : ~{estimated_mass:.2e} Msun")

In [ ]:
red_files = sorted(glob.glob('./M83/M83-Red-*.fit'))
blue_files = sorted(glob.glob('./M83/M83-Blue-*.fit'))
green_files = sorted(glob.glob('./M83/M83-Green-*.fit'))
ha_files = sorted(glob.glob('./M83/M83-HA-*.fit'))

# We will plot the first image of each filter
index = 0
files_to_plot = [red_files[index], blue_files[index], green_files[index], ha_files[index]]

fig, axes = plt.subplots(2, len(files_to_plot), figsize=(15, 5), gridspec_kw={'height_ratios': [2, 1]})

for ax_img, ax_hist, file in zip(axes[0], axes[1], files_to_plot):
    with fits.open(file) as hdulist:
        data = hdulist[0].data

    im = ax_img.imshow(data, cmap='gray', origin='lower', vmin=np.percentile(data,5) , vmax=np.percentile(data,99))
    ax_img.set_title(file.split('-')[1], fontsize=10) 
    ax_img.axis()
    ax_img.grid(False)
    ax_img.axis('off')
    ax_img.set_anchor('C')
    cbar = plt.colorbar(im, ax=ax_img, fraction=0.046, pad=0.04)
    if ax_img == axes[0][-1]:
        cbar.set_label('ADU', fontsize=10)

    # Histogramme
    ax_hist.hist(data.flatten(), bins=1000, color='blue')
    ax_hist.loglog()
    ax_hist.grid(True)
    ax_hist.set_xlabel('ADU', fontsize=10)
    if ax_hist == axes[1][0]:
        ax_hist.set_ylabel('Nombre de pixels', fontsize=10)
plt.tight_layout()
plt.show()


> Quelle est la durée de la pose ? Quelle était la date d’observation ?

> Que pouvez-vous dire de la qualité de cette image ? Quelle est la valeur typique du fond de ciel ?

In [ ]:
fond_ciel = tab[100:200, 100:200]  # Un petit carré dans une région vide
print(fond_ciel.mean()) # ADU

> Répondez aux mêmes questions avec une image dans un autre filtre. Que constatez vous ? Faites bien attention à prendre en compte le temps de pose lorsque vous comparez les flux.

> Convertissez la taille d’un pixel sur l’image en secondes d’arc puis en kiloparsecs, sachant la distance de cette galaxie (se référer au fichier excel TP 2025 galaxies.xlsx). Justifiez. En le mesurant “à la main”, quel est le rayon caractéristique ce cette galaxie, en `kpc` ?

## 3. Calibration des images

Qu’apprend-on ? Vérifiez en particulier le temps d’exposition et la taille de l’image

### Construction de la pose maitre d’offset

Affichez l’histogramme des valeurs des pixels. Autour de quelle valeur se trouvel’offset piédestal ?

In [ ]:
filename = 'Bias/T32-fetedelascience-Bias-000-LD20230411-LT152814-BIN1.fit'
hdul = fits.open(filename)
offset_data_bias = hdul[0].data
hdul.close()

fig, (ax_im, ax_hist) = plt.subplots(1, 2, figsize=(12, 4))

im = ax_im.imshow(offset_data_bias, cmap='gray', origin='lower', 
                   vmin=np.percentile(offset_data_bias, 5), 
                   vmax=np.percentile(offset_data_bias, 99))
ax_im.set_title('Bias (offset)', fontsize=12)
ax_im.axis('off')
ax_im.grid(False)
cbar = fig.colorbar(im, ax=ax_im, fraction=0.046, pad=0.04)
cbar.set_label('ADU', fontsize=10)

ax_hist.hist(offset_data_bias.flatten(), bins=1000, color='blue')
ax_hist.loglog()
ax_hist.set_xlabel('ADU', fontsize=10)
ax_hist.set_ylabel('Nombre de pixels', fontsize=10)
ax_hist.set_title('Histogramme Bias (offset)', fontsize=10)
ax_hist.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# data_dir = "/n07data/laigle/data_to_transf/"
output_dir = "./Processed/"

In [ ]:
list_bias_name = glob.glob('./Bias/*.fit')
print(list_bias_name)

In [ ]:
sigma = 5
master_bias, master_bias_name, master_bias_no_sigma = pr.master_bias(list_bias_name, out_dir=output_dir, 
                                              out_name='masterbias.fits', overwrite=1, sigma=sigma, out_master_bias_no_sigma='masterbias_no_sigma.fits')

In [ ]:
# Assume sigma is already defined before!

# Files to plot
files = [
    ('Processed/masterbias.fits', 'Master Bias avec sigma = ', True),
    ('Processed/masterbias_no_sigma.fits', 'Master Bias sans sigma', False)
]

# Load data first
data_list = []

for filename, _, _ in files:
    with fits.open(filename) as hdul:
        data = hdul[0].data
    data_list.append(data)

# Create figure: 2 rows, 3 columns
fig, axes = plt.subplots(2, 3, figsize=(18, 8), gridspec_kw={'height_ratios': [2, 1]})

# Plot images and individual histograms
for i, (data, (filename, title_prefix, has_sigma)) in enumerate(zip(data_list, files)):
    ax_img = axes[0, i]
    ax_hist = axes[1, i]

    im = ax_img.imshow(data, cmap='gray', origin='lower', 
                       vmin=np.percentile(data, 5), vmax=np.percentile(data, 95))
    fig.colorbar(im, ax=ax_img, fraction=0.046, pad=0.04)
    ax_img.axis('off')
    ax_img.set_anchor('C')
    if has_sigma:
        ax_img.set_title(title_prefix + str(sigma), fontsize=10)
    else:
        ax_img.set_title(title_prefix, fontsize=10)

    ax_hist.hist(data.flatten(), bins=1000, color='blue')
    ax_hist.set_xscale('log')
    ax_hist.set_yscale('log')
    ax_hist.set_xlabel('ADU', fontsize=10)
    ax_hist.set_ylabel('N', fontsize=10)
    if has_sigma:
        ax_hist.set_title('Hist avec sigma = ' + str(sigma), fontsize=10)
    else:
        ax_hist.set_title('Hist sans sigma', fontsize=10)
    ax_hist.grid()

# Superposed histograms (bottom right cell)
colors = ['red', 'green']
labels = ['Avec sigma', 'Sans sigma']
ax_superposed = axes[1, 2]

for data, color, label in zip(data_list, colors, labels):
    ax_superposed.hist(data.flatten(), bins=1000, color=color, label=label)

ax_superposed.set_xscale('log')
ax_superposed.set_yscale('log')
ax_superposed.set_xlabel('ADU', fontsize=10)
ax_superposed.set_ylabel('N', fontsize=10)
ax_superposed.set_title('Superposition des hist', fontsize=10)
ax_superposed.grid()
ax_superposed.legend()

axes[0, 2].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Charger une pose individuelle (exemple de la première image)
bias_example = fits.getdata(list_bias_name[0])

print("---- Pose individuelle ----")
print(f"Moyenne : {np.mean(bias_example):.2f}")
print(f"Ecart-type : {np.std(bias_example):.2f}")

print("---- Master Bias ----")
print(f"Moyenne : {np.mean(master_bias):.2f}")
print(f"Ecart-type : {np.std(master_bias):.2f}")

## 4. Pipeline de traitement

In [ ]:
# Chemin vers le master flat (choisissons Blue par exemple)
master_flat_path = './flats_maitre/master_flat--Blue-T32.fits'

# Ouvrir le fichier FITS
flat_data = fits.getdata(master_flat_path)

# Afficher l'image
plt.figure(figsize=(10, 8))
plt.imshow(flat_data, cmap='gray', origin='lower', vmin=np.percentile(flat_data, 5), vmax=np.percentile(flat_data, 95))
plt.colorbar()
plt.title('Master Flat - Blue')
plt.axis('off')
plt.grid(False)
plt.show()

In [ ]:
# Moyenne des pixels
mean_value = np.mean(flat_data)
print(f"Valeur moyenne des pixels du master flat : {mean_value:.4f}")

In [ ]:
# Tracer l'histogramme
plt.figure(figsize=(8, 6))
plt.hist(flat_data.flatten(), bins=1000, color='blue')
plt.title('Histogramme des valeurs du master flat')
plt.xlabel('Valeur du pixel')
plt.ylabel('Nombre de pixels')
plt.grid(True)
plt.show()


In [ ]:
# 1. Liste des fichiers Blue
blue_images = sorted(glob.glob('./M83/M83-Blue-140-*.fit'))

# 2. Chemins vers master bias et master flat
master_bias = './Processed/masterbias.fits'
master_flat_blue = './flats_maitre/master_flat--Blue-T32.fits'

# 3. Output
output_dir = './Processed/'
out_name = 'M83_Blue_final_calibrated.fits'

# 4. Stack et calibrage
final_blue_image, saved_file = pr.stacking(blue_images, master_bias, master_flat_blue, out_name=out_name, out_dir=output_dir)

print(f"Image calibrée et stackée sauvegardée sous : {saved_file}")

In [ ]:

# Ouvrir une image brute et l'image finale
image_brute = fits.getdata('./M83/M83-Blue-140-001.fit')
image_finale = fits.getdata('./Processed/M83_Blue_final_calibrated.fits')

print("--- Pose individuelle ---")
print(f"Moyenne : {np.mean(image_brute):.2f}")
print(f"Ecart-type : {np.std(image_brute):.2f}")

print("--- Image finale stackée ---")
print(f"Moyenne : {np.mean(image_finale):.2f}")
print(f"Ecart-type : {np.std(image_finale):.2f}")

In [ ]:
# Load the final calibrated image
final_image_data = fits.getdata('./Processed/M83_Blue_final_calibrated.fits')

# Plot the image
plt.figure(figsize=(10, 8))
plt.imshow(final_image_data, cmap='gray', origin='lower',
           vmin=np.percentile(final_image_data, 50),
           vmax=np.percentile(final_image_data, 99))
plt.colorbar(label='ADU')
plt.title('Final Calibrated and Stacked Image - Blue Filter')
plt.axis('off')
plt.grid(False)
plt.show()


In [ ]:
def process_filter(filter_name, input_folder='./M83/', bias_path='./Processed/masterbias.fits',
                   flat_folder='./flats_maitre/', output_folder='./Processed/'):

    # 1. Créer la liste des fichiers pour ce filtre
    pattern = os.path.join(input_folder, f'M83-{filter_name}-*.fit')
    print('pattern', pattern)
    file_list = sorted(glob.glob(pattern))

    if not file_list:
        print(f"Aucun fichier trouvé pour le filtre {filter_name} !")
        return

    print(f"Trouvé {len(file_list)} fichiers pour le filtre {filter_name}.")

    # 2. Trouver le master flat correspondant
    master_flat = os.path.join(flat_folder, f'master_flat--{filter_name}-T32.fits')

    # 3. Définir le nom de l'image finale
    output_name = f'M83_{filter_name}_final_calibrated.fits'

    # 4. Appliquer stacking (calibration + empilement)
    # 1. Master flat
    # 2. Master bias
    final_image, saved_path = pr.stacking(file_list, master_flat, bias_path,
                                          out_name=output_name, out_dir=output_folder)
    
    return final_image, saved_path

def get_initial_image(filter_name, input_folder='./M83/'):
    # 1. Créer la liste des fichiers pour ce filtre
    pattern = os.path.join(input_folder, f'M83-{filter_name}-*.fit')
    file_list = sorted(glob.glob(pattern))

    if not file_list:
        print(f"Aucun fichier trouvé pour le filtre {filter_name} !")
        return None

    # 2. Charger la première image brute
    initial_image = fits.getdata(file_list[0])
    
    return initial_image


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_images_and_histograms(filter_name, color_init='red', color_final='green'):
    """
    Plots the initial and final images and their histograms for a given filter.
    
    Parameters:
        filter_name (str): The filter name (e.g., 'Red', 'Blue', 'Green', 'HA').
        color_init (str): Color for the initial image histogram.
        color_final (str): Color for the final image histogram.
    """
    # Load images
    initial_image = get_initial_image(filter_name)
    final_image, final_path = process_filter(filter_name)
    
    # Create figure and axes
    fig, axes = plt.subplots(2, 2, figsize=(12, 8), gridspec_kw={'height_ratios': [2, 1]})
    
    # Initial image
    im_init = axes[0, 0].imshow(initial_image, cmap='gray', origin='lower',
                                vmin=np.percentile(initial_image, 5),
                                vmax=np.percentile(initial_image, 99))
    fig.colorbar(im_init, ax=axes[0, 0], pad=0.04, fraction=0.046)
    axes[0, 0].set_title(f'Image brute - {filter_name}', fontsize=10)
    axes[0, 0].axis('off')
    
    # Final image
    im_final = axes[0, 1].imshow(final_image, cmap='gray', origin='lower',
                                 vmin=np.percentile(final_image, 5),
                                 vmax=np.percentile(final_image, 99))
    fig.colorbar(im_final, ax=axes[0, 1], pad=0.04, fraction=0.046)
    axes[0, 1].set_title(f'Image finale - {filter_name}', fontsize=10)
    axes[0, 1].axis('off')
    
    # Initial histogram
    axes[1, 0].hist(initial_image.flatten(), bins=1000, color=color_init)
    axes[1, 0].set_xscale('log')
    axes[1, 0].set_yscale('log')
    axes[1, 0].set_xlabel('ADU')
    axes[1, 0].set_ylabel('N')
    axes[1, 0].set_title('Histogramme Image brute', fontsize=10)
    axes[1, 0].grid()
    
    # Final histogram
    axes[1, 1].hist(final_image.flatten(), bins=1000, color=color_final)
    axes[1, 1].set_xscale('log')
    axes[1, 1].set_yscale('log')
    axes[1, 1].set_xlabel('ADU')
    axes[1, 1].set_ylabel('N')
    axes[1, 1].set_title('Histogramme Image finale', fontsize=10)
    axes[1, 1].grid()
    axes[1, 1].set_xlim(1e0, 1e5)
    
    plt.tight_layout()
    plt.show()


In [ ]:
plot_images_and_histograms('Red')
plot_images_and_histograms('Blue')
plot_images_and_histograms('Green')
plot_images_and_histograms('HA')

## 5. Photométrie : taille et luminosité

In [ ]:
# 6.2

from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from photutils.aperture import CircularAperture, aperture_photometry

# Charger ton image finale calibrée
image_data = fits.getdata('./Processed/M83_Red_final_calibrated.fits')  # Exemple avec Red

# Définir le centre de la galaxie (à ajuster si besoin)
x_center = 2048
y_center = 2048
positions = [(x_center, y_center)]

# Définir différentes tailles d'ouverture (en pixels)
radii = [50, 100, 150, 200, 250]  # Rayons en pixels

# Faire des mesures d'ouverture
for r in radii:
    aperture = CircularAperture(positions, r=r)
    phot_table = aperture_photometry(image_data, aperture)
    flux = phot_table['aperture_sum'][0]
    print(f"Rayon : {r} pixels → Flux mesuré : {flux:.2f} ADU")


In [ ]:
# Photométrie simple avec photutils
from photutils.aperture import CircularAperture, aperture_photometry

positions = [(x0, y0)]  # centre de la galaxie
radii = [10, 15, 20, 25, 30]  # à adapter

for r in radii:
    aperture = CircularAperture(positions, r=r)
    phot_table = aperture_photometry(stacked_image, aperture)
    print(f"Rayon {r}: flux = {phot_table['aperture_sum']}")

## 6. Étude statistique loi taille-luminosité

In [ ]:
# Exemple de tracé final
luminosities = [...]  # µJy
radii_kpc = [...]     # kpc
types = [...]         # par exemple ["spirale", "elliptique", ...]

plt.scatter(luminosities, radii_kpc, c=type_color_map[types], label=types)
plt.xlabel("Luminosité (µJy)")
plt.ylabel("Rayon effectif (kpc)")
plt.title("Relation taille-luminosité")
plt.legend()
plt.grid()
plt.show()

## 7. Discussion

- Incertitudes principales :
  * fond de ciel
  * étalonnage du point-zéro
  * qualité des poses
- Comportement selon le type morphologique ?
- Alignement avec les résultats de Gadotti (2009) ?